## Imports

In [15]:
import pandas as pd
import requests
import json
import os

## Get financial statements

In [4]:
# читаем выгрузку из реестра
d1 = pd.read_excel('./данные/data1.xlsx', header=8)
d2 = pd.read_excel('./данные/data2.xlsx', header=8)

In [80]:
# обрабатываем случай 'один продукт - несколько компаний'
otm1 = d1[d1['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].str.replace(';', '').str.split('\n').sum()

otm2 = d2[d2['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].str.replace(';', '').str.split('\n').sum()

In [92]:
# обрабатываем случай 'один продукт - одна компания'
oto1 = d1[~d1['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].astype(str).values

oto2 = d2[~d2['Идентификационный номер (ИНН)'].str.contains('\n', na=False)
         ]['Идентификационный номер (ИНН)'].astype(str).values

In [ ]:
# получаем список уникальных организаций
inns = list(set(otm1) | set(otm2) | set(oto1) | set(oto2))
api_key = ''

# получаем данные фин отчетности, локально сохраняем
for inn in inns:
    x = requests.get(f'https://api.checko.ru/v2/finances?key={api_key}&inn={inn}')
    with open(f'./reports/{inn}.txt', 'w') as f:
        json.dump(x.json(), f)

## Parse financial statements

In [16]:
# путь до фин отчетности
root = './reports/'
# годы, за которые смотрим фин результат
years = set(['2020', '2021', '2022', '2023'])

companies = pd.DataFrame()

for filename in os.listdir(root):
    with open(root + filename, 'r') as f:
        data = json.load(f)
        
    # для исследования нужна только выручка, ее код 2110
    revenue = {}

    for year in data['data']:
        if year in years and '2110' in data['data'][year]:
            revenue[year] = data['data'][year]['2110']
    
    if len(years - set(revenue.keys())) > 0:
        for year in years - set(revenue.keys()):
            revenue[year] = 0
    
    if 'company' in data:
        data['company'].update(revenue)
        if companies.empty:
            companies = pd.DataFrame(data['company'], index=[0])
        else:
            companies = pd.concat([companies, pd.DataFrame(data['company'], index=[0])])

In [17]:
companies.to_csv('./parsed_data.csv', index=False)

## Prepare registry

In [9]:
pd.concat([d1, d2]).reset_index(drop=True).to_csv('./registry.csv', index=False)